This is a test file that can be ignored.

In [1]:
import pandas as pd
import nfl_data_py as nfl
import os
import urllib.request
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns
from matplotlib.offsetbox import AnnotationBbox, OffsetImage
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score


In [16]:
# Prompt the user to enter a year
year_input = input("Enter the year: ")

# Convert the input to an integer
try:
    year = int(year_input)
except ValueError:
    print("Invalid input. Please enter a valid year.")
    exit()

In [17]:
#load the nfl data
df_year = nfl.import_pbp_data([year])
df_teams = nfl.import_team_desc()
#df_year.head #unecessary step; done to check that data loaded, readable

2022 done.
Downcasting floats.


In [18]:
#some filtering

df_year = df_year[df_year["season_type"] == "REG"] #filter to regular season
df_year = df_year[(df_year['pass']==1) | (df_year['rush'] == 1)]  #get rid of plays where there is only one team on the field (like kick offs)
df_year = df_year.dropna(subset = ['epa', 'posteam', 'defteam']) #get rid of plays without epa, a possession team, or a defense team
df_year.shape
df_year.head

<bound method NDFrame.head of        play_id          game_id old_game_id home_team away_team season_type  \
2         68.0  2022_01_BAL_NYJ  2022091107       NYJ       BAL         REG   
3         89.0  2022_01_BAL_NYJ  2022091107       NYJ       BAL         REG   
4        115.0  2022_01_BAL_NYJ  2022091107       NYJ       BAL         REG   
5        136.0  2022_01_BAL_NYJ  2022091107       NYJ       BAL         REG   
7        202.0  2022_01_BAL_NYJ  2022091107       NYJ       BAL         REG   
...        ...              ...         ...       ...       ...         ...   
47828   3520.0  2022_18_TEN_JAX  2023010701       JAX       TEN         REG   
47830   3544.0  2022_18_TEN_JAX  2023010701       JAX       TEN         REG   
47831   3577.0  2022_18_TEN_JAX  2023010701       JAX       TEN         REG   
47834   3651.0  2022_18_TEN_JAX  2023010701       JAX       TEN         REG   
47836   3677.0  2022_18_TEN_JAX  2023010701       JAX       TEN         REG   

       week posteam p

In [ ]:
# Function to calculate mean EPA for first down in the given year
def calculate_mean_epa_down1(year):
    import nfl_data_py as nfl
    df_year = nfl.import_pbp_data([year])
    mean_epa_down1 = df_year[df_year['down'] == 1].groupby('posteam')['epa'].mean().reset_index()
    mean_epa_down1['Rank'] = mean_epa_down1['epa'].rank(ascending=False, method='min')
    mean_epa_down1 = mean_epa_down1[['Rank', 'posteam', 'epa']].sort_values('epa', ascending=False)
    return mean_epa_down1